In [ ]:
import requests ## Importation des librairies nécessaires
import bs4
import pandas as pd

url_home = "https://pokemondb.net/pokedex/national"

response = requests.get(
    url_home,
    headers={"User-Agent": "Python for data science tutorial"}
)

page = bs4.BeautifulSoup(response.content, "lxml")

print(page.title.text)

In [ ]:
## Fonction qui télécharge la page HTML d’un Pokémon et la transforme en objet BeautifulSoup
def get_name(nom_pokemon): ## Remplir le nom du pokemon voulu
    url = "https://pokemondb.net/pokedex/" + nom_pokemon
    response = requests.get(url)
    return bs4.BeautifulSoup(response.text, "lxml")

In [ ]:
## Test en affichant le titre de la page bulbasaur
page_bulba = get_name("bulbasaur")
print(page_bulba.title.text)

In [ ]:
## Fonction extraire_stats qui permet d'extraire toutes les infos des 4 tableaux sur la page du pokemon
def extraire_stats(page_pokemon):
    tableaux = page_pokemon.find_all("table", {"class": "vitals-table"})[:4] 
    stats = {} ## Création d'un dictionnaire vide 

    for table in tableaux:
        lignes = table.find_all("tr")
        for ligne in lignes:
            th = ligne.find("th")  # nom de la ligne
            td = ligne.find("td")  # valeur

            if th and td:
                nom = th.text.strip()
                valeur = td.text.strip()
                stats[nom] = valeur

    return stats ## Retourne dans le dictionnaire stats

In [ ]:
## Test avec la page de bulbasaur
stats_bulba = extraire_stats(page_bulba)
stats_bulba

In [ ]:
liens = page.find_all("a", {"class": "ent-name"}) ## Liens récupère toute les balises "a" pour avoir tous les noms de pokemon
liste_noms = [a.text.lower() for a in liens] ## Liste_noms permet d'avoir la liste de tous les noms en minuscule
print(liens)             
print(liste_noms[:10])   # limite aux 10 premiers pour pas tout afficher
print(len(liste_noms))   # Voir si le nombre total est correct

In [ ]:
## Cette fonction récupère toutes les informations d'un pokemon à partir de son nom 
def infos_pokemon(nom):
    page = get_name(nom)
    stats = extraire_stats(page)
    stats["name"] = nom       # on ajoute le nom du pokémon
    return stats

In [ ]:
liste_finale = []  ## Création d'une liste pour récupérer le dictionnaire de chaque pokemon

for nom in liste_noms[:10]:  ## Boucle qui récupère les informations des 10 premiers pokemon
    print("Récupération pour", nom) 
    liste_finale.append(infos_pokemon(nom)) ##Ajoute les données dans liste_finale en exécutant la fonction infos_pokemon

print(liste_finale) ## Vérification 

In [ ]:
df = pd.DataFrame(liste_finale)
df

In [ ]:
!pip install scikit-image
!pip install matplotlib

In [ ]:
import shutil
import os

from skimage import io
import matplotlib.pyplot as plt

In [ ]:
dossier = "images_pokemon" ## Création d'un dossier pour mettre les photos

if not os.path.exists(dossier): 
    os.makedirs(dossier)

In [ ]:
def telecharger_image(nom_pokemon):
    url = f"https://img.pokemondb.net/artwork/{nom_pokemon}.jpg" ## Permet de récupérer le lien de l'image grace au nom de chaque pokemon
    chemin = f"{dossier}/{nom_pokemon}.jpg"  ## Indique le chemin où l'image est placée 

    r = requests.get(url) ## Télécharge le contenu de l'url 

    if r.ok: 
        with open(chemin, "wb") as f: ## L'image est placée dans le dossier images-pokemon
            f.write(r.content)
        print("Image téléchargée :", chemin) ## Si le téléchargement c'est bien passé dire image téléchargé avec le chemin
    else:
        print("Erreur :", nom_pokemon) ## Sinon Erreur avec le nom du pokemon

    return chemin

In [ ]:
premiers = liste_noms[:5] ## Prend uniquement les 5 premiers pokemons 
chemins_images = [] ## Création d'une liste pour stocker le chemin des images téléchargées

for nom in premiers: ## Création d'une boucle qui ajoute l'image des 5 premiers pokemons dans le dossier images_pokemon
    chemin = telecharger_image(nom) 
    chemins_images.append(chemin) 


In [ ]:
plt.figure(figsize=(15, 4))

for i, chemin in enumerate(chemins_images):
    img = io.imread(chemin)
    plt.subplot(1, 5, i+1)
    plt.imshow(img)
    plt.axis("off")

plt.show()
print("Les premiers pokemon du Pokedex")